## Предсказание законтрактованности путём агрегации данных о предыдущих поставках

Мы проведём сравнение двух методов:
1) Усреднение параметров поставок (таких как процент опоздания по поставкам и изменение качества)
2) Усреднение эмбеддингов предыдущих поставок, полученных через LSTM автоэнкодер

In [1]:
import numpy as np
import pandas as pd

from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

import sys
import os
sys.path.append(os.path.abspath('..'))
from data.feature_generation import pipeline
from figures.plots import show_embeddings
from src.metrics import mapk

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pipeline()
data.head(2)

,supplier,id,spec_date,delivery_period_end,bids_contracted,is_late,is_underweight,is_poorquality,supplier_status,option,...,6,7,8,9,10,11,12,13,14,15
0,50 ЛЕТ ОКТЯБРЯ ООО,"50 ЛЕТ ОКТЯБРЯ ООО_Юг Руси, АО_2022-12-19",2022-12-19,2022-12-25,1.0,0.0,1.0,1.0,1.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50 ЛЕТ ОКТЯБРЯ ООО,"50 ЛЕТ ОКТЯБРЯ ООО_Юг Руси, АО_2023-01-24",2023-01-24,2023-02-12,1.0,0.0,0.0,0.0,1.0,15.0,...,0.510517,0.245941,-0.13836,-0.16784,0.107102,-0.216089,-0.284308,-0.345988,-0.072292,-0.333858


In [23]:
"""
Разделение набора данных
"""

split_point = np.quantile(data['spec_date'], 0.9)
train_spec = data.loc[data['spec_date'] < split_point]
test_spec = data.loc[data['spec_date'] >= split_point]

X_train = train_spec.drop(['bids_contracted', 'is_late', 'is_underweight', 'is_poorquality'], axis=1)
y_train = train_spec['bids_contracted']

X_test = test_spec.drop(['bids_contracted', 'is_late', 'is_underweight', 'is_poorquality'], axis=1)
y_test = test_spec['bids_contracted']

features = {'zpp4': ['supplier_lateness', 'supplier_underweight', 'supplier_price_change'],
            'embed': [str(i) for i in range(16)],
            'spec': ['mean_delivery_length', 'delivery_length_diff', 'mean_volume', 'volume_diff', 'conversion']}

## Предсказание законтрактованности независимо от даты

In [14]:
"""
Предсказание законтрактованности (Простые агрегации)
"""

def estimate(model, X_test, y_test):
    accuracy = accuracy_score(y_test, model.predict(X_test))
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    print(f'Accuracy: {accuracy}\nROC AUC: {roc_auc}')

print('Бустинг (Агрегации поставок):')
model = CatBoostClassifier(verbose = 0, random_state=42).fit(X_train[features['zpp4']], y_train)
estimate(model, X_test[features['zpp4']], y_test)

Бустинг (Простые агрегации):
Accuracy: 0.6047904191616766
ROC AUC: 0.6295572916666665


In [16]:
"""
Предсказание законтрактованности (Эмбеддинг)
"""

print('Бустинг (эмбеддинг):')
model = CatBoostClassifier(verbose = 0, random_state=42).fit(X_train[features['embed']], y_train)
estimate(model, X_test[features['embed']], y_test)

Бустинг (агрегация эмбеддингов):
Accuracy: 0.6317365269461078
ROC AUC: 0.6715921336206896


## Предсказание законтрактованности отдельно по дням

In [17]:
def dayseparately(X, y, features, predict_func):
    actual = []
    predicted = []
    
    days_indexes = X.groupby('spec_date').apply(lambda x: x.index)
    for day in days_indexes:
        actual.append(list(y.loc[day].values))
        predicted.append(list(predict_func(X[features].loc[day])))
    
    return actual, predicted

In [21]:
print('Бустинг (Агрегации поставок):')
model = CatBoostClassifier(verbose = 0, random_state=42).fit(X_train[features['zpp4']], y_train)
actual, predicted = dayseparately(X_test, y_test, features['zpp4'], lambda x: model.predict(x))
print(f'{mapk(actual, predicted)}')

Бустинг (Простые агрегации):
0.277136351808483


In [22]:
print('Бустинг (Эмбеддинг):')
model = CatBoostClassifier(verbose = 0, random_state=42).fit(X_train[features['embed']], y_train)
actual, predicted = dayseparately(X_test, y_test, features['embed'], lambda x: model.predict(x))
print(f'{mapk(actual, predicted)}')

Бустинг (агрегация эмбеддингов):
0.27628415300546444


## Комбинирование переменных

In [26]:
"""
Предсказание законтрактованности (Простые агрегации + агрегации спецификаций)
"""

model = CatBoostClassifier(verbose = 0, random_state=42).fit(X_train[features['zpp4'] + features['spec']], y_train)
estimate(model, X_test[features['zpp4'] + features['spec']], y_test)

Бустинг (Простые агрегации):
Accuracy: 0.6586826347305389
ROC AUC: 0.7257543103448275


In [27]:
"""
Предсказание законтрактованности (Эмбеддинг + агрегации спецификаций)
"""

model = CatBoostClassifier(verbose = 0, random_state=42).fit(X_train[features['embed'] + features['spec']], y_train)
estimate(model, X_test[features['embed'] + features['spec']], y_test)

Бустинг (агрегация эмбеддингов):
Accuracy: 0.6497005988023952
ROC AUC: 0.7206941451149426


## Комбинирование переменных (отдельно по дням)

In [28]:
"""
Предсказание законтрактованности (Простые агрегации + агрегации спецификаций)
"""

model = CatBoostClassifier(verbose = 0, random_state=42).fit(X_train[features['zpp4'] + features['spec']], y_train)
actual, predicted = dayseparately(X_test, y_test, features['zpp4'] + features['spec'], lambda x: model.predict(x))
print(f'{mapk(actual, predicted)}')

0.3014133923150316


In [29]:
"""
Предсказание законтрактованности (Эмбеддинг + агрегации спецификаций)
"""

model = CatBoostClassifier(verbose = 0, random_state=42).fit(X_train[features['embed'] + features['spec']], y_train)
actual, predicted = dayseparately(X_test, y_test, features['embed'] + features['spec'], lambda x: model.predict(x))
print(f'{mapk(actual, predicted)}')

0.29336499262728766
